<a href="https://colab.research.google.com/github/avats-dev/ml-dl-challenges/blob/main/HE-DL/HE_DL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
#!unzip '/content/drive/MyDrive/datasets/data-he.zip'

In [39]:
%cd /content/dataset/
!ls

/content/dataset
test  train  train.csv


In [40]:
df = pd.read_csv(r"/content/dataset/train.csv")
df.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [44]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
dataframe=df,
directory="/content/dataset/train",
x_col="Image",
y_col="Class",
subset="training",
batch_size=16,
class_mode="categorical",
target_size=(224,224))

val_generator=datagen.flow_from_dataframe(
dataframe=df,
directory="/content/dataset/train",
x_col="Image",
y_col="Class",
subset="validation",
batch_size=16,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))


Found 4852 validated image filenames belonging to 6 classes.
Found 1617 validated image filenames belonging to 6 classes.


In [45]:
base = tf.keras.applications.VGG19(
    include_top=False,
    input_shape=(224,224,3),
    weights='imagenet'
)
for layer in base.layers:
    layer.trainable=False

base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [46]:
from tensorflow.keras import layers

x = layers.Flatten()(base.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(6, activation='softmax')(x)

vggmodel = tf.keras.models.Model(base.input, x)
vggmodel.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [47]:
vggmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [48]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
vgghist = vggmodel.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=val_generator,
                              validation_steps=STEP_SIZE_VALID,
                              epochs=20)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
303/303 [==============================] - 20s 61ms/step - loss: 2.6467 - accuracy: 0.4177 - val_loss: 1.0508 - val_accuracy: 0.6683
Epoch 2/20
303/303 [==============================] - 18s 59ms/step - loss: 1.1354 - accuracy: 0.5869 - val_loss: 0.9405 - val_accuracy: 0.6702
Epoch 3/20
303/303 [==============================] - 18s 60ms/step - loss: 1.0864 - accuracy: 0.6298 - val_loss: 0.9131 - val_accuracy: 0.6869
Epoch 4/20
303/303 [==============================] - 18s 60ms/step - loss: 0.9782 - accuracy: 0.6758 - val_loss: 0.9095 - val_accuracy: 0.6813
Epoch 5/20
303/303 [==============================] - 18s 60ms/step - loss: 0.9148 - accuracy: 0.6846 - val_loss: 0.8483 - val_accuracy: 0.7042
Epoch 6/20
303/303 [==============================] - 18s 60ms/step - loss: 0.8488 - accuracy: 0.7138 - val_loss: 0.8398 - val_accuracy: 0.7135
Epoch 7/20
303/303 [==============================] - 18s 60ms/step - loss: 0.8040 - accuracy: 0.7330 - val_loss: 0.9050 - val_accuracy:

In [50]:
import os

TEST_DIR = '/content/dataset/test/'
test_images = os.listdir(TEST_DIR)
predictions = []

for image in test_images:
    img = tf.keras.preprocessing.image.load_img(TEST_DIR + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (224, 224))
    img = tf.reshape(img, (-1, 224, 224, 3))
    print (vggmodel.predict(img/255))
    predictions.extend(vggmodel.predict(img/255).argmax(axis = 1))

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
prediction = [labels[k] for k in predictions]

[[0.3285025  0.02023481 0.04506603 0.01322062 0.5551378  0.03783822]]
[[0.0688731  0.0222843  0.06547504 0.0269754  0.7890147  0.02737742]]
[[0.21426511 0.02583848 0.05482365 0.02088104 0.6439439  0.0402478 ]]
[[2.6665205e-06 2.5715628e-06 1.1002001e-01 8.8115197e-01 8.5604480e-03
  2.6234682e-04]]
[[9.3897549e-11 1.2616027e-07 9.9999321e-01 6.2930936e-09 6.2958165e-06
  3.7223563e-07]]
[[0.09998554 0.16157372 0.0957773  0.04001724 0.49669027 0.105956  ]]
[[1.9139452e-04 1.2905632e-04 6.6666946e-02 8.6480385e-01 6.5328963e-02
  2.8798187e-03]]
[[1.1715779e-04 1.6959129e-03 9.6595895e-01 3.6377565e-04 2.8812969e-02
  3.0512901e-03]]
[[1.9497781e-10 3.2910009e-07 9.9998844e-01 1.3968698e-08 1.0617470e-05
  5.9330847e-07]]
[[0.07644863 0.18606468 0.12078638 0.03680254 0.47048864 0.10940912]]
[[0.10262693 0.05805226 0.09006997 0.04048803 0.65050215 0.05826072]]
[[8.9533430e-01 1.9646520e-04 4.4464171e-03 6.6885207e-04 9.7192109e-02
  2.1618470e-03]]
[[0.08193022 0.21270996 0.09058657 0.035

In [51]:
sub = pd.DataFrame({'image_id': test_images, 'label': prediction})
display(sub)
sub.to_csv('submission.csv', index = False)

,image_id,label
0,image750.jpg,Miscellaneous
1,image3305.jpg,Miscellaneous
2,image9446.jpg,Miscellaneous
3,image4309.jpg,Jacket
4,image8279.jpg,Christmas_Tree
...,...,...
3484,image5106.jpg,Miscellaneous
3485,image5045.jpg,Miscellaneous
3486,image7718.jpg,Miscellaneous
3487,image8225.jpg,Christmas_Tree
